In [ ]:
from pathlib import Path
import pickle
from typing import Any

import numpy as np
from datasets import load_dataset, Dataset
from tqdm.auto import tqdm
from scipy.special import log_softmax
import IPython.display
import lhotse
from lhotse import RecordingSet, Recording, AudioSource, CutSet, SupervisionSegment

In [33]:
dataset = load_dataset('yodas2_ru000_32k', data_files='*.parquet', split='train').with_format('np')

Resolving data files:   0%|          | 0/331 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/331 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [37]:
def yodas_hf_sample_to_cut(sample: dict[str, Any], data_dir: str = '.') -> lhotse.cut.Cut:
    if 'path' in sample['audio']:
        recording = Recording.from_file(
            data_dir + '/' + sample['audio']['path'],
            recording_id=sample['video_id']
        )
    elif 'bytes' in sample['audio']:
        recording = Recording.from_bytes(
            sample['audio']['bytes'],
            recording_id=sample['video_id']
        )
    else:
        raise NotImplementedError('use .cast_column("audio", decode=False)')
    cut = recording.to_cut()
    utterances = sample['utterances']
    cut.supervisions = [
        SupervisionSegment(
            id=utt_id,
            recording_id=recording.id,
            start=start,
            duration=end - start,
            text=text,
        )
        for start, end, text, utt_id
        in zip(utterances['start'], utterances['end'], utterances['text'], utterances['utt_id'])
    ]
    return cut

print(yodas_hf_sample_to_cut(dataset[0], data_dir='yodas2_ru000_32k'))

MonoCut(id='YoSwHP0w-so', start=0.0, duration=51.873416666666664, channel=0, supervisions=[SupervisionSegment(id='YoSwHP0w-so-00000-00000000-00000726', recording_id='YoSwHP0w-so', start=0.0, duration=7.26, channel=0, text='Привет друзья и с вами снова Жека у себя на балконе и Давайте опять послушаем короткие', language=None, speaker=None, gender=None, custom=None, alignment=None), SupervisionSegment(id='YoSwHP0w-so-00001-00000726-00001278', recording_id='YoSwHP0w-so', start=7.26, duration=5.5199995, channel=0, text='анекдот от меня да да друзья вам не послышалось именно от меня от вашего', language=None, speaker=None, gender=None, custom=None, alignment=None), SupervisionSegment(id='YoSwHP0w-so-00002-00001278-00001872', recording_id='YoSwHP0w-so', start=12.78, duration=5.9399996, channel=0, text='Джеки Поэтому заранее ставим такой вот жирный лайкос Можно два и подписываемся', language=None, speaker=None, gender=None, custom=None, alignment=None), SupervisionSegment(id='YoSwHP0w-so-0000